# Исследование надёжности заёмщиков

**Заказчик** - <em>кредитный отдел банка.</em><br>
**Данные** - <em>статистика о платёжеспособности клиентов.</em><br>
**Цель** - <em>разобраться, влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.</em> <br>
Результаты проекта будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.</p>

#### Описание данных
- children - количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Оглавление
1. [Загрузка данных и изучение общей информации по проекту](#download-data)
2. [Предобработка данных](#data-preprocessing)
- [Обработка пустых значений](#processing-empty-values)
- [Замена типа данных](#сhange-type)
- [Обработка дубликатов](#processing-duplicates)
- [Лемматизация данных](#lemmatization)
- [Категоризация данных](#categorization)
3. [Анализ данных](#data-analyse)
5. [Выводы](#conclusions)

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
Отсутствие оглавления.
<br>1. Для чего нужно: 1 – это некий план согласно которого ты выполняешь задачу, 2 – проверяющему или пользователю легче ориентироваться в работе.
<br>2. Оглавление должно быть интерактивным – нажимаем –> переходим к тому или иному пункту решения поставленной задачи.
<br>3. Отсутствует описание данных.
</div>

## <a id='download-data'><span style="color:black">1. Загрузка данных и изучение общей информации по проекту</span></a>

Изучение данных с помощью текствого редактора позволило определить столбец 'Unnamed: 0' в качестве индекса, что позволило оптимизировать используемую память, 

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('data.csv', index_col='Unnamed: 0')
display(data)
display(data.describe())
display(data.info(memory_usage='deep'))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 11.4 MB


None

<p>Предварительное изучение данных позволило выявить следующие аномалии данных:</p>

- минимальный возраст 0 (скорее всего это связанос с неправильным внесением данных в базу), 
- минимальное количество детей -1 (скорее всего это связанос с неправильным внесением данных в базу), 
- максимальное количество детей 20 (скорее всего это связано с неправильным внесением данных в базу), 
- в столбце education текстовые данные имееют разный регистр (скорее всего это связано с неправильным внесением данных в базу),
- данные из столбца days_employed не репрезетантивны (содержат, в том числе, и отрицательные значения) - возможно возникла ошибка при передаче данных, информацию нужно уточнить у специалистов, отвечавших за сбори и выгрузку данных, 
- данные из столбца total_income следует преобразовать к целочисленному типу для удобства чтения.

<p>Воспользовавшись методом info можно заметить, что количество не пустых значений в столбцах 'days_employed' и 'total_income' отличается от общего количества значений в других столбцах.</p>

**Вывод:** Изучение данных позволило выявить ряд недостатков данных, которые необходимо устранить до перехода к анализу.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. sample/head/tail, describe, info лучше все в одной ячейке и используй display.
<br>2. Когда делаешь выводы, старайся делать и предположения почему так произошло или какова причина. Почему 20 детей? Многодетная семья, сомнительно. Скорее всего ошиблись при вводе. Опять же можно посмотреть, это единичное значение или нет.
Проверяющему будет проще проследить ход мыслей и поправить/подсказать.
<br>3. Что за столбец Unnamed: 0. Перед загрузкай всегда смотри файл с помощью текстового редактора, чтобы иметь представление что ты загружаешь, тогда при загрузке можешь указать типы для явных данных (только если уверен на 99%) или указать столбец с индексом.
</div>

## <a id='data-preprocessing'><span style="color:black">1. Предобработка данных</span></a>

### <a id='processing-empty-values'><span style="color:black">Обработка пустых значений</span></a>

Подсчитаем количество пропущенных значений в базе данных.

In [2]:
display(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. В ячейках кода только пояснение к коду. Никаких комментариев по выполнению работы в ячейках с кдом не должно быть, для этого есть Markdown ячейки.
</div>

Общее количество пропущенных значений в столбцах 'days_employed' и 'total_income' равно 2174, что составляет более 10% от общего количества данных в соответсвующих категориях. Так как удаление такого большого объема данных может повлиять на дальнейший анализ, необходимо произвести замену пустых значений в данных столбцах.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. Из практики - 10% это не много, если данные однообразны (необходимо обосновать) и их больше ~100 000, то можно пожертвовать, если ограничено время на исследование данных. Но если стоит задача поиска аномалий - категорически нельзя удалять. Но лучше конечно пропуски заполнять.
</div>

Проведем более подробное изучение данные в каждом столбце таблицы по отдельности. Проверим строки в которых имеется хотя бы одно пустое значение

In [3]:
display(data[data.isnull().any(axis=1)].sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14887,0,NaN,47,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,приобретение автомобиля
14065,0,NaN,63,Среднее,1,женат / замужем,0,F,сотрудник,0,NaN,получение образования
20675,0,NaN,66,СРЕДНЕЕ,1,вдовец / вдова,2,F,пенсионер,0,NaN,покупка своего жилья
11200,2,NaN,34,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,покупка коммерческой недвижимости
13892,1,NaN,50,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилья для сдачи


Проверка строк с пропусками показывает взаимосвязь между трудовым стажем и месячным доходом (данные в соответствующих столбцах отсутствуют в одних и тех же строках). Возможно это связано с тем, что клиентом не предоставлена справка о доходах в которой обычно отражается период работы в организации.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Неплохое предположение.
</div>

In [4]:
display(data[(data.days_employed >= 0)]['income_type'].value_counts())

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [5]:
display(data.income_type.value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Выборочная проверка данных позволила выдвинуть теорию о том, что трудовой стаж находится в зависимости от типа занятости. Дальнейшая проверка показала, что только две катогории имеют положительно значение в столбце days_employed: пенсионеры и безработные, т.е. люди которые на момент предоставления информация не работают, остальные категории имеют отрицательное значение и пустое. В рамках текущего проекта обработать данные по столбцу days_employed и привести их к репрезентативному виду не представляется возможным и необходимым (так как эти данные не влияют на исследование). Можно предположить, что некорректные данные появились во время выгрузки данных, но при этом сохранена определенная зависимость. Таким образом при заполнении данных необходимо учитывать тип занятости, а столбец 'days_employed' можно удалить для оптимизации используемой памяти.

In [6]:
data = data.drop('days_employed', axis=1)
categories = data['income_type'].unique()
for i in categories:
    data.loc[(data['total_income'].isna()) & (data['income_type'] == i), 'total_income'] \
    = data.loc[data['income_type'] == i, 'total_income'].median()
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   dob_years         21525 non-null  int64  
 2   education         21525 non-null  object 
 3   education_id      21525 non-null  int64  
 4   family_status     21525 non-null  object 
 5   family_status_id  21525 non-null  int64  
 6   gender            21525 non-null  object 
 7   income_type       21525 non-null  object 
 8   debt              21525 non-null  int64  
 9   total_income      21525 non-null  float64
 10  purpose           21525 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


None

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. Заполнение медианным значением в зависимости от категорий - простой, но достаточно эффективный способ. Однако у тебя зависисмость только от категорий одного критерия, можно сделать больее сложную, вложенную структуру. Есть еще более эффективные способы. Но на текущий момент это самый оптимальный.
<br>2. !!!! Придерживайся принципа - действие + отображение в ОДНОЙ ячейке.
</div>

Проверка показывает, что пропущенные значения отсутствуют.

In [7]:
display(data.children.value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [34]:
print(f'Количество клиентов в возрасте до 25 лет, имеющих 20 детей: \
{data[(data.children == 20) & (data.dob_years <= 25)]["dob_years"].value_counts().sum()}')

Количество клиентов в возрасте до 25 лет, имеющих 20 детей: 0


Проверка данных по стабцу children показывает, что cтроки содержащие значение от '6' до '19' отсутствуют, что косвенно свидетельствует на ошибку ввода. Цифра 20 в категории 'Количество детей' была указана ошибочно, так как такие данные были указаны в том числе по отношении к клиентам, которые не могли иметь такое количество детей в силу возраста. Скорее всего данные были не правильно занесены в базу и вместо '20' должно быть указано '2', а вместо '-1' - '1'. При этом количество строк, содержащих значение '-1' составляет меньше процента от количества семей с одним ребенком, поэтому такие данные допустимо удалить. Частотность значени '20' составляет 3,7% от количество семей с 2 детьми, поэтому такие значения целесообразно изменить на '2'. 

In [9]:
data = data[data['children'] != -1]
data.loc[data['children'] == 20, 'children'] = 2
display(data.children.value_counts())

0    14149
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Я бы тоже так сделал исходя из моего комментария выше. Тут хорошо.
</div>

Кредит не может быть выдан несовершеннолетему, поэтому проверим столбец 'dob_years' на наличие таких значений.

In [10]:
display(data[(data.dob_years < 18)]['dob_years'].value_counts())

0    101
Name: dob_years, dtype: int64

Проверяем данные со значением dob_years равное нулю на предмет возможных закономерностей.

In [11]:
display(data[(data.dob_years == 0)].sample(20))

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20577,0,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
99,0,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
6859,1,0,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,128246.646591,покупка недвижимости
15433,0,0,среднее,1,в разводе,3,F,сотрудник,0,73509.183779,приобретение автомобиля
15295,0,0,среднее,1,Не женат / не замужем,4,F,сотрудник,0,49082.259548,автомобиль
7075,1,0,высшее,0,Не женат / не замужем,4,M,компаньон,0,161439.581182,на покупку своего автомобиля
16861,0,0,среднее,1,в разводе,3,F,пенсионер,0,218284.757642,покупка жилья
2870,0,0,среднее,1,женат / замужем,0,F,компаньон,0,167713.212064,на покупку своего автомобиля
10384,1,0,высшее,0,женат / замужем,0,F,сотрудник,0,153478.301459,сделка с подержанным автомобилем
17613,0,0,высшее,0,женат / замужем,0,F,сотрудник,0,114124.233190,покупка коммерческой недвижимости


В столбце dob_years значение '0' встречается 101 раз, при этом какие-либо закономерности, которые могут влиять на такое значение не выявлены: таким клиентам соответсвуют разные значения в категориях "образование" (есть в том числе и данные о клиентах с высшим образование, чей возраст не может быть менее 20 лет), "дети", "семейное положение", "пол", "тип занятости" и "цель полочения кредита". Так как общее количество строк, содержащих такое значение не велико (менее 0,5%), и сформировать какую-то определенную выборку не возможно, то такие строки не могут повлиять на дальнейшее исследование и могут быть удалены.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Как я говорил, удаляем, если данные однообразны, но это надо обосновать. Иначе, допустим, все представители одной категории некого критерия оказались в той выборке, которую ты удалил. Следовательно ты не сможешь описать, сделать выводы и т.д. по этой категории, а клиент не зная ничего о этой категории, может потерять выгоду.
</div>

In [12]:
data = data[data['dob_years'] != 0]
display(data.dob_years.describe())

count    21377.000000
mean        43.499509
std         12.248988
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Аномалия с возрастом клиентов устранена из базы данных.

In [13]:
display(data['gender'].value_counts())

F      14129
M       7247
XNA        1
Name: gender, dtype: int64

Проверка данных в столбце gender показывает наличие неиндефицируемого значения 'XNA'. С учетом того, что такое значение единично его можно было бы удалить, однако такое значение можно и заменить на другое (на исследование это не повлияет). Так как значение категориальное и представлено всего двумя видами данных будет целесообразным заменить его на самое распространенное, т.е. на значение моды.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>В таких случаях обычно описывают категории (F и M), и сравнивают с неизвестным значением или можно воспользоваться машинным обучением (задачи классификации). Поскольку здесь одно значение, можно сделаить так, как ты сделал, или удалить.
</div>

In [14]:
display(data.gender.mode())

0    F
Name: gender, dtype: object

In [15]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
display(data['gender'].value_counts(dropna=False))

F    14130
M     7247
Name: gender, dtype: int64

**Вывод:** Важно знать каким образом формируются данные, для того чтобы найти подходящий метод замены случайных и не случайных пропусков. Случайные пропуски могут быть удалены.

### <a id='processing-duplicates'><span style="color:black">Обработка дубликатов</span></a>

In [16]:
display(data['education'].value_counts())

среднее                13660
высшее                  4678
СРЕДНЕЕ                  766
Среднее                  706
неоконченное высшее      665
ВЫСШЕЕ                   272
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Изучение данных стоблца education показывает наличие дубликатов: имеются одинаковые категории с разным регистром. Чтобы учесть такие дубликаты, все символы в строке приводим к нижнему регистру.

In [17]:
data['education'] = data['education'].str.lower()
display(data['education'].value_counts())

среднее                15132
высшее                  5216
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверка показывает, что после обработки дубликаты в столбце education отсутствуют.

In [18]:
display(data['family_status'].value_counts(dropna=False))

женат / замужем          12302
гражданский брак          4151
Не женат / не замужем     2792
в разводе                 1181
вдовец / вдова             951
Name: family_status, dtype: int64

В столбце family_status особых замечание по хранящемся в нем значений нет. Дубликатов и пропусков данных нет.

**Вывод:** Дубликаты слов с разным регистров появляются по причине отсутствия единых стандартов внесения данных в базу данных. Есть смысл обратиться к специалистам, разрабатыющим программное обеспечение для ввода данных в базу с целью определения стандартов ввода данных.

### <a id='сhange-type'><span style="color:black">Заменяем типы данных</span></a>

Для дальнейшего удобства представления данных поменяем их тип в столбце total_income.

In [19]:
data['total_income'] = data['total_income'].astype('int')
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21377 non-null  int64 
 1   dob_years         21377 non-null  int64 
 2   education         21377 non-null  object
 3   education_id      21377 non-null  int64 
 4   family_status     21377 non-null  object
 5   family_status_id  21377 non-null  int64 
 6   gender            21377 non-null  object
 7   income_type       21377 non-null  object
 8   debt              21377 non-null  int64 
 9   total_income      21377 non-null  int32 
 10  purpose           21377 non-null  object
dtypes: int32(1), int64(5), object(5)
memory usage: 1.9+ MB


None

Для оптимизации использования памяти проведем соответствующий анализ по типу числовых данных.

In [20]:
int_types = ["int8", "int16", "int32", "int64"]
float_types = ["float16", "float32", "float64"]
for it in int_types:
    print(np.iinfo(it))
for it in float_types:
    print(np.finfo(it))

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =   

В Python есть различные подтипы данных для хранения целых и вещественных чисел, которые могут использовать меньшее число байт для представления каждого значения. Применим функцию для оптимизации использования памяти для хранения числовых данных исходя из их минимальных и максимальных значений, а также изменим тип данных столбцов, хранящих данные типа object, в которых уникальными являются менее 50% значений. Если все значения в столбце уникальны, то использование типа category приведёт к повышению уровня использования памяти.

In [21]:
def optimize_memory_usage(df, print_size=True):
    numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    before_size = df.memory_usage().sum() / 1024**2    
    for column in df.columns:
        column_type = df[column].dtypes
        if column_type in numerics:
            column_min = df[column].min()
            column_max = df[column].max()
            if str(column_type).startswith('int'):
                if column_min > np.iinfo(np.int8).min and column_max < np.iinfo(np.int8).max:
                    df[column] = df[column].astype(np.int8)
                elif column_min > np.iinfo(np.int16).min and column_max < np.iinfo(np.int16).max:
                    df[column] = df[column].astype(np.int16)
                elif column_min > np.iinfo(np.int32).min and column_max < np.iinfo(np.int32).max:
                    df[column] = df[column].astype(np.int32)
                elif column_min > np.iinfo(np.int64).min and column_max < np.iinfo(np.int64).max:
                    df[column] = df[column].astype(np.int64)  
            elif str(column_type).startswith('float'):
                if column_min > np.finfo(np.float16).min and column_max < np.finfo(np.float16).max:
                    df[column] = df[column].astype(np.float16)
                elif  column_min > np.finfo(np.float32).min and column_max < np.finfo(np.float32).max:
                    df[column] = df[column].astype(np.float32)
                else:
                    df[column] = df[column].astype(np.float64)
        elif column_type == 'object':
            num_unique_values = len(df[column].unique())
            num_total_values = len(df[column])
            if num_unique_values / num_total_values < 0.5:
                df[column] = df[column].astype('category')
                

    after_size = df.memory_usage().sum() / 1024**2
    if print_size: print('Memory usage size: before {:5.4f} Mb - after {:5.4f} Mb ({:.1f}%).'.format(before_size, after_size, 100 * (before_size - after_size) / before_size))
    return df
optimize_memory_usage(data)
display(data.info())
display(data.memory_usage())

Memory usage size: before 1.8756 Mb - after 0.4507 Mb (76.0%).
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21377 non-null  int8    
 1   dob_years         21377 non-null  int8    
 2   education         21377 non-null  category
 3   education_id      21377 non-null  int8    
 4   family_status     21377 non-null  category
 5   family_status_id  21377 non-null  int8    
 6   gender            21377 non-null  category
 7   income_type       21377 non-null  category
 8   debt              21377 non-null  int8    
 9   total_income      21377 non-null  int32   
 10  purpose           21377 non-null  category
dtypes: category(5), int32(1), int8(5)
memory usage: 461.5 KB


None

Index               171016
children             21377
dob_years            21377
education            21589
education_id         21377
family_status        21589
family_status_id     21377
gender               21501
income_type          21741
debt                 21377
total_income         85508
purpose              22753
dtype: int64

Замена подтипов числовых данных и данных, имевших значение типа object, позволила значительно уменьшить потребление памяти при этом сами данные отображаются корректно.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Есть типы данных int32, float32 и другие. Посмотри возможные варианты. Выполни небольшой анализ по зависимости memory usage от типов данных. 
</div>

**Вывод:** Замена типов данных в отдельных или во всех столбцах базы данных позваляет значительно уменьшить объем памяти и соответственно увеличить скорость обработки данных, что особенно важно при анализе больших наборов данных.

### <a id='lemmatization'><span style="color:black">Лемматизация данных</span></a>

In [22]:
data['purpose'].value_counts(dropna=False)

свадьба                                   792
на проведение свадьбы                     772
сыграть свадьбу                           768
операции с недвижимостью                  671
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     648
операции с жильем                         646
жилье                                     640
покупка жилья для семьи                   640
покупка жилья                             638
строительство собственной недвижимости    632
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          620
строительство недвижимости                620
покупка своего жилья                      618
покупка недвижимости                      618
ремонт жилью                              609
покупка жилой недвижимости                602
на покупку своего автомобиля              501
заняться высшим образованием      

Изучение столбца purpose показавает большое количество одних и тех же категорий записанных по разному. Для обработки таких данных проведем над ними лемматизацию.

In [23]:
from pymystem3 import Mystem
m = Mystem()

text = ' '.join(data['purpose'].unique())
lemmas = m.lemmatize(text)
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

**Вывод:** Данные в столбце purpose имееют ограниченное количество уникальных данных: значительно меньше общего количества значений в данном столбце. При этом имеется большое количество слов, имеющих один корень, или схожих по смыслу. Это связано с отсутвием стандартов для ввода данных этой категории в базу данных. Задать такой стандарт не всегда возможно (зависит, в том числе, от объема данных), но обсудить внедрение стандартов с разработчиками ПО стоит. При этом, если такие недостатки данных встречаются, лемматизация позволяет эффективно обрабатывать текстовые наборы данных.

### <a id='categorization'><span style="color:black">Категоризация данных</span></a>

На основе списка лемматизированных слов осуществим категоризацию данных: появятся 4 категории, которые будут охватывать все ранее представленные.

In [24]:
def purpose_group(lemmas):
    for word in lemmas.split(' '): 
        if 'недвижимост' in word or 'жил' in word:
            return 'недвижимость'
        if 'авто' in word:
            return 'автомобиль'
        if 'свадьб' in word:
            return 'свадьба'      
    return 'образование'

data['purpose'] = data['purpose'].apply(purpose_group)
data['purpose'].value_counts(dropna=False)

недвижимость    10769
автомобиль       4282
образование      3994
свадьба          2332
Name: purpose, dtype: int64

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Тут все верно. 
</div>

**Выводы**: Так как провести эффективный анализ данных с первоначальными значениями в столбце purpose не представляется возможным (фактически несколько разных значений внесенных в базу данных относится к одной и той же категории), то решить данную проблему позволяет категоризация данных.

После всех проведенных манипуляций с данными проведем проверку на дубликаты с помощью метода .duplicated()

In [25]:
display(data.duplicated().sum())

404

In [26]:
data = data.drop_duplicates()
display(data.duplicated().sum())

0

Выявленные дубликаты были удалены с помощью метода .drop_duplicates().

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Если days_employed не информативен, то его стоило сразу удалить после сделанного вывода.
</div>

## <a id='data-analyse'><span style="color:black">3. Анализ данных</span></a>

Для изучения зависимости возврата кредита от различных факторов воспользуемся функцией создания сводной таблицы данных.

In [33]:
def pivot_table(data, index):
    data_pivot = data.pivot_table(index=index, values='debt', aggfunc=['count', 'sum'])
    data_pivot.columns = ['total_amount', 'debt']
    data_pivot['debt %'] = data_pivot['debt'] * 100 / data_pivot['total_amount']
    data_pivot = data_pivot.fillna(0)
    return data_pivot

Отобразим процентное соотношение возврата кредита в зависимости от наличия/отсутствия детей.

In [28]:
display(pivot_table(data, 'children'))

,total_amount,debt,debt %
children,,,
0,13761,1056,7.673861
1,4735,441,9.313622
2,2101,202,9.614469
3,327,27,8.256881
4,40,4,10.000000
5,9,0,0.000000


**Вывод:** Наблюдается прямая зависимость между наличием детей и погашением кредитных обязательств в срок: клиенты без детей меньше подвержены рискам невозврата кредита, имеют больший доход, приходящийся на каждого члена семьи (при ее наличии), не имееют затрат, связанных с воспитанием и содержанием детей.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Я бы не сказал, что клиенты без детей более "ответсвенно" относятся к выплатам по кредиту, я бы сказал что они меньше подвержены рискам, связанным с наличем детей.
</div>

Отобразим процентное соотношение возврата кредита в зависимости от семейного положения.

In [29]:
display(pivot_table(data, 'family_status'))

,total_amount,debt,debt %
family_status,,,
Не женат / не замужем,2763,273,9.880565
в разводе,1179,85,7.209500
вдовец / вдова,934,62,6.638116
гражданский брак,4098,386,9.419229
женат / замужем,11999,924,7.700642


**Вывод:** Наблюдается прямая зависимость между наличием или отсутствием супруга/супруги (в том числе если если клиент ранее был в браке) и выплатой кредита в срок. Не женатые/замужние клиенты менее подвержены рискам не возврата кредитов. Как и в случае с детьми это может быть связано с большим доходом, приходящимся на 1 человека, а также меньшими потребностями одинокого (не семейного) человека.

Для определения зависимости между уровнем доходов и наличием задолженности по кредиту проведем категоризацию по уровню доходу. Разабьем данные на 3 примерно равные группы со средним доходом, ниже и выше среднего.

In [30]:
data['income_level'] = pd.qcut(data['total_income'],
                              q=3,
                              labels=['ниже среднего', 'средний', 'выше среднего'])

In [31]:
display(pivot_table(data, 'income_level'))

,total_amount,debt,debt %
income_level,,,
ниже среднего,6991,578,8.267773
средний,7010,626,8.930100
выше среднего,6972,526,7.544464


**Вывод:** Выборка показывает, что клиенты с более высоким уровнем имеют больше возможностей для погашения кредита при равных затратах, а также такие люди более финасово грамотны и умеют распоряжатся своими деньгами.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Аналогично с комментарием выше. "Дисциплинированы" лучше не употреблять. Я бы написал так. У кого выше доход, у тех больше возможностей погасить кредит. Также обычно такие люди более финасово грамотны и умеют распоряжатся своими деньгами.
</div>

Отобразим процентное соотношение исполнения кредитных обязательств в зависимости целей на целей кредита.

In [32]:
display(pivot_table(data, 'purpose'))

,total_amount,debt,debt %
purpose,,,
автомобиль,4239,398,9.389007
недвижимость,10508,778,7.403883
образование,3936,370,9.400407
свадьба,2290,184,8.034934


**Вывод:** Кредиты, выданные на приобретение недвижимости или организацию свадьбы, являются более надежными. 
Это может быть связано со следующими причинами: 
- кредиты на приобретение недвижимости обычно выдают под залог этого имущества, что побуждает клиентов своевременно погашать кредиты чтобы не потерять собственность;
- своевременное погашение кредитов организацию свадьбы возможно связано с эмоцианальной составлющей - если такой кредит является краткосрочным существует меньше рисков его не погашения, т.к. на первый год совместной жизни приходится наименьшее количество расходов согласно статистике (разведеные не очень заинтересованы в погашении такого кредита);
- погашение кредитов на автомобиль связан с рисками утери такого имущества, например в ДТП или в результата угона;
- кредитование на цели образования связанос с рисками утраты клиентом интереса к получению такого образования.

## <a id='conclusions'><span style="color:black">4. Выводы</span></a>

В ходе выполнения проекта были решены следующие задачи. 

- При предобработке данных были выявлены пустые значения в категориях 'трудовой стаж' и 'доход в месяц', которые могли быть не внесены изначально в базу из-за не предоставления соответсвующих сведений клиентом. Данные из столбца 'total_income' были заменены медианным значением, а столбец days_employed был удален из базы данных, т.к. не мог повлиять на исследование (стаж работы в организации практически не влияет на возможность выплаты по кредиту).
- С помощью ручного поиска с учетом регистра были выявлены дубликаты в категории 'образование', что свидетельствует о некорректном вводе данных в базу. Все данные в указанной категории были приведены к одному регистру. Перед началом анализа данных был применен метод .duplicated, который позволил дополнительно выявить дубликаты, появившиеся в результате преобработки данных.
- С помощью метода .astype() был изменен тип данных значений в столбцах базы данных, что позволило значительно уменьшить объем используемой памяти. 
- Лемматизацию данных столбца purpose провели с помощью библиотеки pymystem3. 
- Для этого столбца сделали категоризацию, т.к. изначально одни и теже категории были записаны разными словами. Кроме того, была проведана категоризация по уровню дохода, чтобы в последствии применить новые категории в анализе зависимости между уровнем дохода и возвратом кредита.

Анализ предоставленных данных показал, что неженатый клиент, не имеющий детей, но с высоким уровнем дохода, берущий кредит на организацию свадьбы, наиболее предпочтителен для кредитной организации.


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. Выводы всегда и везде выводы делай в виде списка. А так "каша" получается.
<br>2. Выводы по работе офрмляй так, чтобы это было понятно, что это выводы. В том числе и по тексту. Выводы имеют свой стиль формления, комментарии по работе свой. Но не такое офрмление, что выколи глаз (не надо разноцветных текстов и т.д.). Используй списки, Bold, Italic и т.д.
<br>3. Выводы старайся делать после кадого этапа (этапы согласно оглавлению), кратко что сделал и основные выводы. В конце работы проделанные этапы и ключевые выводы.
<br>4. Про вдавца исправь на неженатый)   
<br>5. Мои комментарии не удаляй!!!
</div>